In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_last_month():
    today = datetime.today()
    first = today.replace(day=1)
    last_month = first - timedelta(days=1)
    return last_month.strftime("%Y%m")

base_url = 'https://olinda.bcb.gov.br/olinda/servico/MPV_DadosAbertos/versao/v1/odata/MeiosdePagamentosMensalDA(AnoMes=@AnoMes)?@AnoMes=%27201601%27&$format=json&$select=AnoMes,quantidadePix,valorPix,quantidadeTED,valorTED,quantidadeTEC,valorTEC,quantidadeCheque,valorCheque,quantidadeBoleto,valorBoleto,quantidadeDOC,valorDOC'

year_month = get_last_month()

# URL completa com o mês anterior
url = base_url.format(ano_mes)

# Faz a requisição GET para buscar os dados
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    # Converte a resposta JSON para um DataFrame
    df = pd.DataFrame(data['value'])
    
    # Carregar dados existentes, se houver
    try:
        df_existing = pd.read_csv('meios_pagamentos_mensal.csv')
        # Concatenar os dados novos aos existentes
        df_combined = pd.concat([df_existing, df]).drop_duplicates(subset=['AnoMes']).reset_index(drop=True)
    except FileNotFoundError:
        # Se o arquivo não existir, use apenas os novos dados
        df_combined = df
    
    column_mapping = {
            "AnoMes": "YearMonth",
            "quantidadePix": "quantityPix",
            "valorPix": "valuePix",
            "quantidadeTED": "quantityTED",
            "valorTED": "valueTED",
            "quantidadeTEC": "quantityTEC",
            "valorTEC": "valueTEC",
            "quantidadeCheque": "quantityBankCheck",
            "valorCheque": "valueBankCheck",
            "quantidadeBoleto": "quantityBrazilianBoletoPayment",
            "valorBoleto": "valueBrazilianBoletoPayment",
            "quantidadeDOC": "quantityDOC",
            "valorDOC": "valueDOC"
    }

        # Renomeia as colunas do DataFrame
    df_combined.rename(columns=column_mapping, inplace=True)
    
    # Salva os dados combinados em um arquivo CSV
    df_combined.to_csv('meios_pagamentos_mensal.csv', index=False)
    
    
    # Exibe os primeiros registros do DataFrame
    print(df_combined.head())
else:
    print(f"Erro na requisição: {response.status_code}")
